Los datos para trabajar:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

url = 'https://raw.githubusercontent.com/apimentelaUP/recursos/master/mushrooms/mushrooms.csv'
tabla_hongos = pd.read_csv(url)

La tarea es hacer un clasificador Bayesiano para predecir si un hongo es comestible.

In [ ]:
tabla_hongos

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,e,?,s,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,e,?,s,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,e,?,s,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,t,?,s,k,w,w,p,w,o,e,w,v,l


### Naive

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(tabla_hongos,test_size=0.2)

In [ ]:
frecuencia_clases = train.groupby(["class"]).size()

In [ ]:
apriori = frecuencia_clases/len(train)

apriori

class
e    0.51531
p    0.48469
dtype: float64

In [ ]:
X = train.drop(columns=["class"])
clases = apriori.index.values

presunciones = {}

for clase in clases:
  presunciones[clase] = {}

for caracteristica in X.columns:
  frecuencia_caracteristicas = train.groupby(["class",caracteristica]).size()

  for clase in clases:
    presunciones[clase][caracteristica]= frecuencia_caracteristicas[clase]/frecuencia_clases[clase]

In [ ]:
presunciones["e"]["odor"]

odor
a    0.096745
l    0.096148
n    0.807107
dtype: float64

In [ ]:
test

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
7137,p,k,s,e,f,y,f,c,n,b,t,?,s,s,w,p,p,w,o,e,w,v,d
6091,p,f,y,n,f,y,f,c,n,b,t,?,s,k,w,p,p,w,o,e,w,v,l
6819,p,f,s,n,f,y,f,c,n,b,t,?,k,k,p,w,p,w,o,e,w,v,d
2562,e,x,y,e,t,n,f,c,b,w,t,b,s,s,p,w,p,w,o,p,n,v,d
4527,p,x,y,y,f,f,f,c,b,g,e,b,k,k,n,n,p,w,o,l,h,v,d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5542,p,f,y,y,f,f,f,c,b,g,e,b,k,k,n,n,p,w,o,l,h,y,d
3232,p,x,s,w,f,c,f,w,n,u,e,b,s,s,w,w,p,w,o,p,n,s,d
5931,p,x,y,n,f,y,f,c,n,b,t,?,s,s,p,w,p,w,o,e,w,v,p
6744,e,x,f,g,f,n,f,w,b,g,e,?,k,k,w,w,p,w,t,p,w,s,g


In [ ]:
import math

X = test.drop(columns=["class"])
salida = []

for i,evento in X.iterrows():
  logprob_clases = {clase: math.log(apriori[clase]) for clase in clases}

  for caracteristica, valor in evento.iteritems():
    for clase in clases:
      try:
        logprob_clases[clase] += math.log(presunciones[clase][caracteristica][valor])
      except:
        logprob_clases[clase] = -float("Inf")
    
  print(i,logprob_clases)

## SKLEARN

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

codificador_entrada = OneHotEncoder()
codificador_salida = LabelEncoder()

X = codificador_entrada.fit_transform(train.drop(columns=["class"]))
y = codificador_salida.fit_transform(train["class"])

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clasificador = MultinomialNB()
clasificador.fit(X,y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
X = codificador_entrada.transform(test.drop(columns=["class"]))

clasificador.predict(X)

array([1, 1, 1, ..., 1, 0, 0])